# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install gymnasium[box2d] swig box2d --quiet
!pip install cma --quiet
!pip install stable-baselines3[extra] gymnasium[box2d]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 3.3 MB/s eta 0:00:00


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [ ]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [ 0.00249033  1.4095557   0.25222024 -0.06064852 -0.00287881 -0.05713172
  0.          0.        ] -> Action: 0
Obs: [ 0.00498056  1.4076138   0.25187948 -0.08631674 -0.00570194 -0.0564677
  0.          0.        ] -> Action: 1
Obs: [ 0.00740128  1.4050617   0.24316117 -0.11343178 -0.00677426 -0.02144809
  0.          0.        ] -> Action: 0
Obs: [ 0.00982208  1.4019096   0.24316366 -0.14010327 -0.00784726 -0.02146171
  0.          0.        ] -> Action: 0
Obs: [ 0.01224289  1.3981574   0.24316709 -0.16677204 -0.00891971 -0.02145072
  0.          0.        ] -> Action: 3
Obs: [ 0.01475143  1.3938015   0.2541591  -0.19362296 -0.01219594 -0.06553073
  0.          0.        ] -> Action: 0
Obs: [ 0.01726017  1.3888454   0.2541694  -0.22029194 -0.01547071 -0.06550128
  0.          0.        ] -> Action: 0
Obs: [ 0.0197689   1.3832896   0.25417942 -0.24696109 -0.01874546 -0.06550074
  0.          0.        ] -> Action: 3
Obs: [ 0.02236624  1.3771333   0.2653072  -0.27370057 -0.02424877

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [ ]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [ ]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00449982  1.4089478  -0.45581022 -0.08767305  0.00522107  0.10324769
  0.          0.        ] -> Action: 0
Obs: [-0.00900021  1.4063982  -0.4551937  -0.11334014  0.01032294  0.1020476
  0.          0.        ] -> Action: 0
Obs: [-0.01350079  1.403249   -0.45520955 -0.14000888  0.01542183  0.10198708
  0.          0.        ] -> Action: 0
Obs: [-0.01800146  1.3995001  -0.4552246  -0.16668177  0.02052009  0.10197435
  0.          0.        ] -> Action: 0
Obs: [-0.02250233  1.3951515  -0.45523953 -0.1933522   0.02561753  0.10195826
  0.          0.        ] -> Action: 0
Obs: [-0.02700338  1.3902031  -0.45525455 -0.22002226  0.03071415  0.10194196
  0.          0.        ] -> Action: 0
Obs: [-0.03150453  1.3846551  -0.45526952 -0.24669243  0.03580998  0.1019261
  0.          0.        ] -> Action: 0
Obs: [-0.03600588  1.3785075  -0.45528445 -0.27336273  0.040905    0.10190995
  0.          0.        ] -> Action: 0
Obs: [-0.04050741  1.3717601  -0.45529932 -0.30003315  0.04599924 

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [ ]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to save models/logs on Google Drive
save_dir = "/content/drive/MyDrive/lunarlander_models"

Mounted at /content/drive


In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback


# Create environment
env = gym.make("LunarLander-v3")

# Define agent
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    policy_kwargs=dict(net_arch=[64, 64]),
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99
)

# Create evaluation environment
eval_env = gym.make("LunarLander-v3")

# Setup evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=f"{save_dir}/best_model",  # save best model
    log_path=f"{save_dir}/logs",                   # save logs
    eval_freq=10_000,                              # evaluate every 10k steps
    deterministic=True,
    render=False
)

# Train the agent with callback
model.learn(total_timesteps=200_000, callback=eval_callback)

# Save final model to Drive
model.save(f"{save_dir}/ppo_lunarlander_final")

env.close()
eval_env.close()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91.7     |
|    ep_rew_mean     | -207     |
| time/              |          |
|    fps             | 618      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 90.8         |
|    ep_rew_mean          | -195         |
| time/                   |              |
|    fps                  | 627          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0057583638 |
|    clip_fraction        | 0.00688      |
|    clip_range           | 0.2          |
|    en

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-280.47 +/- 43.94
Episode length: 268.60 +/- 72.27
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 269         |
|    mean_reward          | -280        |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.022259051 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.35       |
|    explained_variance   | -7.28e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 241         |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0109     |
|    value_loss           | 447         |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96.5     |
|    ep_rew_mean     | -164 

In [ ]:
# Path to best model in Google Drive
best_model_path = "/content/drive/MyDrive/lunarlander_models/best_model/best_model.zip"

# Load the best model
model = PPO.load(best_model_path)

# Re-create environment for evaluation
env = gym.make("LunarLander-v3")

n_eval_episodes = 100
rewards = []

for _ in range(n_eval_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward
    rewards.append(total_reward)

print(f"Average test reward over {n_eval_episodes} episodes: {np.mean(rewards):.2f} ± {np.std(rewards):.2f}")

env.close()

Average test reward over 100 episodes: -68.31 ± 59.87
